# Historical Australian temperature and rainfall anomalies  
This script is intended to read temperature and rainfall files from CMIP6 historical models.  
It will mask out all areas except for Australia and Eastern Australia. 
It will calulate anomalies on a monthly and seasonal basis and plot them as a time series.
It will also produce a time averaged map of the australian region with gridlines and labels etc.

Lets start by making this script for just one file and then I'll try calling this script in another notebook which loops over all the different historical model run files. 

In [2]:
# load all the packages I will need
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import cartopy.crs as ccrs # to add in continents and change map projections 
from matplotlib.colors import LinearSegmentedColormap # to change colour bar????



In [3]:
# load in data files (I need to figure out how to loop over models and add in multiple files)
pr_ds = xr.open_dataset('/g/data/lp01/CMIP6/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/Amon/pr/gr1.5/v20181126/pr_Amon_BCC-CSM2-MR_historical_r1i1p1f1_gr1.5_185001-201412.nc')
ts_ds = xr.open_dataset('/g/data/lp01/CMIP6/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/Amon/ts/gr1.5/v20181126/ts_Amon_BCC-CSM2-MR_historical_r1i1p1f1_gr1.5_185001-201412.nc')

In [6]:
# import land fraction data
landfrac = xr.open_dataset('/home/563/kb6999/Models/landfraction_file_grid1.5.nc')

In [10]:
landfrac


<xarray.Dataset>
Dimensions:  (lat: 120, lon: 240, time: 1)
Coordinates:
  * lon      (lon) float64 0.0 1.5 3.0 4.5 6.0 ... 352.5 354.0 355.5 357.0 358.5
  * lat      (lat) float64 -89.25 -87.75 -86.25 -84.75 ... 86.25 87.75 89.25
  * time     (time) datetime64[ns] 2001-01-01T01:00:00
Data variables:
    data     (time, lat, lon) float32 1.0 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0
Attributes:
    CDI:          Climate Data Interface version 1.6.0 (http://code.zmaw.de/p...
    Conventions:  CF-1.0
    history:      Sun May 12 08:58:19 2013: cdo gtc,0.3 fractional_land.1.5-d...
    CDO:          Climate Data Operators version 1.6.0rc5 (http://code.zmaw.d...

In [7]:
# change units 
pr = pr_ds.pr *86400 #convert units to mm/day
ts = ts_ds.ts - 273 # convert units from K to C

In [8]:
# here I select the Australia region and then mask out the ocean
pr_aus = pr.sel(lat=slice(-50,-5), lon=slice(110,160)) # these two lines select out a box containing australia
ts_aus = ts.sel(lat=slice(-50,-5), lon=slice(110,160))

# these two lines replace any ocean grid sections with nan so only land sections have a value
pr_aus = pr_aus.where(landfrac>0) 
ts_aus = ts_aus.where(landfrac>0)

In [13]:
# just testing that the land masking is working.  The resolution is uper low and bad though and idk why really but try using Jo's other dataset first before getting too stressed. 
ax = plt.pcolormesh.axes(projection=ccrs.PlateCarree()) # plot results with coastlines

ts_aus.mean(dim='time').plot(ax=ax, transform=ccrs.PlateCarree(), cmap='coolwarm')
ax.coastlines(resolution='50m')
ax.set_extent([110,160,-50,-5]) # sets axis bounds to show only australia region lon, lon, lat, lat
# add on grid lines for longitude and latitude at specified range and spacing
ax.gridlines(xlocs=range(110,160,10), ylocs=range(-50,-10,10),draw_labels=False) 
ax.gridlines(xlocs=range(110,160,10), ylocs=range(-50,-10,10),draw_labels=True)
# add in different grid lines for tropics
tropics = ax.gridlines(ylocs=[-66.5,-23.43691,23.43691,66.5],draw_labels=False,linewidth=2,linestyle='--', edgecolor='dimgrey')
tropics.xlines=False

#labels for latitude and longitude
ax.text(0,-0.07,'Longitude', transform=ax.transAxes, ha='left')
ax.text(-0.05,0,'Latitude', transform=ax.transAxes, rotation='vertical', va='bottom')

AttributeError: 'function' object has no attribute 'axes'

In [ ]:
# group the data into months
pr_aus_mon = pr_aus.groupby('time.month')
ts_aus_mon = ts_aus.groupby('time.month')
# calculate the mean climatology along each month for the time period 1850-1900 
pr_aus_clim = pr_aus.sel(time=slice('1850-01','1900-12')).groupby('time.month').mean(dim='time')
ts_aus_clim = ts_aus.sel(time=slice('1850-01','1900-12')).groupby('time.month').mean(dim='time')

In [ ]:
# caclulate the anomalies for each month
pr_anom_mon = (pr_aus_mon - pr_aus_clim)
ts_anom_mon = (ts_aus_mon - ts_aus_clim)



In [ ]:
# to use functions:
import sys
sys.path.append('')
import anomaly_function as f_anom

In [ ]:
f_anom.monthly_anom(pr_aus, 1850-11, 1900-12)

In [ ]:
# still tryin to get a nice bar graph
# plot the precipitation anomalies for each month throughout the time period
# need to make the figure size really big so that all data points are included
rain_series = pr_anom_mon.sel(time=slice('1850-01','1900-12')).mean(dim=('lat','lon')).to_series() # convert to series so I can make a bar graph, but then I think it might be elimating data here

f = plt.figure(figsize=(100, 60))

ax = f.add_subplot(111)

#subplot1
rain_series.where(rain_series>0).plot.bar(ax=ax, color = 'blue')
rain_series.where(rain_series<0).plot.bar(ax=ax, color = 'red')
# add gridlines
ax.grid(which='major', linestyle='-', linewidth='0.5', color='black') # customise major grid
ax.set_axisbelow(True) # Don't allow the axis to be on top of your data, don't necessarily need this line
ax.minorticks_on() # need this line in order to get the minor grid lines 
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# axis labels and limits
ax.set_xticks([]) #disable xticks
ax.set_ylabel('Precipitation anomaly [mm/day]', fontsize=60)
ax.set_ylim(ymin = -3, ymax = 4)

In [ ]:
# still tryin to get a nice bar graph
# plot the precipitation anomalies for each month throughout the time period
# need to make the figure size really big so that all data points are included
rain_series = pr_anom_mon.sel(time=slice('1850-01','1900-12')).mean(dim=('lat','lon')).to_series() # convert to series so I can make a bar graph, but then I think it might be elimating data here

f = plt.figure(figsize=(100, 60))

ax = f.add_subplot(211)
ax2 = f.add_subplot(212)

#subplot1
rain_series.where(rain_series>0).plot.bar(ax=ax, color = 'blue')
rain_series.where(rain_series<0).plot.bar(ax=ax, color = 'red')
# add gridlines
ax.grid(which='major', linestyle='-', linewidth='0.5', color='black') # customise major grid
ax.set_axisbelow(True) # Don't allow the axis to be on top of your data, don't necessarily need this line
ax.minorticks_on() # need this line in order to get the minor grid lines 
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# axis labels and limits
ax.set_xticks([]) #disable xticks
ax.set_ylabel('Precipitation anomaly [mm/day]', fontsize=60)
ax.set_ylim(ymin = -3, ymax = 5)

#subplot 2
rain_series2 = pr_anom_mon.sel(time=slice('1850-01','1900-12')).mean(dim=('lat','lon'))

rain_series2.plot(ax=ax2, color = 'blue')
# add gridlines
ax2.grid(which='major', linestyle='-', linewidth='0.5', color='black') # customise major grid
ax2.set_axisbelow(True) # Don't allow the axis to be on top of your data, don't necessarily need this line
ax2.minorticks_on() # need this line in order to get the minor grid lines 
ax2.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# axis labels and limits
#ax.set_xticks([]) #disable xticks
ax2.set_ylabel('Precipitation anomaly [mm/day]')
ax2.set_ylim(ymin = -3, ymax = 5)

In [ ]:
print(rain_series)

In [ ]:
print(pr_anom_mon.mean(dim=('lat','lon')))

In [ ]:
# plot the precipitation anomalies for each month throughout the time period
fig, ax = plt.subplots() # make a figure

pr_anom_mon.mean(dim=('lat','lon')).plot(color = 'blue', ax=ax) #plot the anomalies
plt.title('Monthly precipitation anomalies', fontsize=16)
plt.ylabel('Precipitation anomaly [mm/day]') # this overrides the label which comes on automatically using the xarray title

ax.grid(which='major', linestyle='-', linewidth='0.5', color='black') # customise major grid
ax.minorticks_on() # need this line in order to get the minor grid lines 
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')

fig.set_figheight(8) # these two parameters change the figure height and width 
fig.set_figwidth(15)
#fig.suptitle('example figure title', fontsize=16)


In [ ]:
# plot the precipitation anomalies for each month throughout the time period
fig, ax = plt.subplots() # make a figure

ts_anom_mon.mean(dim=('lat','lon')).plot(color = 'red', ax=ax) #plot the anomalies
plt.title('Monthly temperature anomalies', fontsize=16)
plt.ylabel('Temperature anomaly [C]') # this overrides the label which comes on automatically using the xarray title

ax.grid(which='major', linestyle='-', linewidth='0.5', color='black') # customise major grid
ax.minorticks_on() # need this line in order to get the minor grid lines 
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')

fig.set_figheight(8) # these two parameters change the figure height and width 
fig.set_figwidth(15)
#fig.suptitle('example figure title', fontsize=16)

In [ ]:
# just chekcing the climatology and anomalies are being calculated correctly
pr_aus_mon.mean(dim=('lat','lon')).plot(size=6)
plt.title('Monthly precipitation')
pr_aus_clim.mean(dim=('lat','lon')).plot(size=6)
plt.title('Average monthly precipitation from 1850-1900')
pr_anom_mon.mean(dim=('lat','lon')).plot(size=6)
plt.title('Monthly precipitation anomalies')


In [ ]:
# these next few cells are to calculate seasonal anomalies
# first I need to define a new coordinate (seasonyear) so that december gets counted with the adjoining jan and feb
seasonyear_pr = (pr_ds.time.dt.year + (pr_ds.time.dt.month//12)) #
pr_ds.coords['seasonyear'] = seasonyear_pr
seasonyear_ts = (ts_ds.time.dt.year + (ts_ds.time.dt.month//12))
ts_ds.coords['seasonyear'] = seasonyear_ts

In [ ]:
# define function to calculate the seasonal mean:
def seasonal_mean(data):
    return data.groupby('time.season').mean()

In [ ]:
#then you can select a season:
pr_djf = pr_yearly_seasonal.sel(season='DJF')
# calculate the mean climatology along each season for the time period 1850-1900
pr_clim_seasonal = pr_yearly_seasonal.sel(seasonyear=slice('1850-01','1900-12')).mean(dim='seasonyear')
ts_clim_seasonal = ts_yearly_seasonal.sel(seasonyear=slice('1850-01','1900-12')).mean(dim='seasonyear')
#calculate the anomaly and extract the pr variable out of the xarray so pr_anom_seasonal is not an xarray
pr_anom_seasonal = (pr_yearly_seasonal.pr - pr_clim_seasonal.pr)*86400 # convert units again to mm/day
ts_anom_seasonal = (ts_yearly_seasonal.ts - ts_clim_seasonal.ts) # anomaly so no need to change temp units


In [ ]:
# select the anomaly for each season
# precipitation
pr_anom_djf = pr_anom_seasonal.sel(season = 'DJF')
pr_anom_mam = pr_anom_seasonal.sel(season = 'MAM')
pr_anom_jja = pr_anom_seasonal.sel(season = 'JJA')
pr_anom_son = pr_anom_seasonal.sel(season = 'SON')

# temperature
ts_anom_djf = ts_anom_seasonal.sel(season = 'DJF')
ts_anom_mam = ts_anom_seasonal.sel(season = 'MAM')
ts_anom_jja = ts_anom_seasonal.sel(season = 'JJA')
ts_anom_son = ts_anom_seasonal.sel(season = 'SON')

In [ ]:
# again do some plots so I can check it's calculating right 
pr_djf = pr_yearly_seasonal.pr.sel(season='DJF')
pr_djf.mean(dim=('lat','lon')).plot(size=6)
pr_clim_djf = pr_clim_seasonal.pr.sel(season='DJF')
pr_clim_djf.mean(dim=('lat','lon')).plot(size=6)

In [ ]:
# make a subplot for the graphs
# if you do subplots this way then I can adjust the size of them

f = plt.figure(figsize=(15,10))
ax = f.add_subplot(221)
ax2 = f.add_subplot(222)
ax3 = f.add_subplot(223)
ax4 = f.add_subplot(224)

#subplot 1 DJF
pr_anom_djf.mean(dim=('lat','lon')).plot(ax=ax, color ='red')
# add gridlines
ax.grid(which='major', linestyle='-', linewidth='0.5', color='black') # customise major grid
ax.set_axisbelow(True) # Don't allow the axis to be on top of your data, don't necessarily need this line
ax.minorticks_on() # need this line in order to get the minor grid lines 
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# change axis labels
ax.set(xlabel=None)
ax.set(ylabel='Precipitation anomaly [mm/day]')
# set axis limits
ax.set_ylim(ymin= -0.05, ymax=0.07)
#ax.set_xlim(self, bottom=None, top=None, emit=True, auto=False, *, ymin=None, ymax=None)

# subplot 2 MAM
pr_anom_mam.mean(dim=('lat','lon')).plot(ax=ax2, color='orange')
# add gridlines
ax2.grid(which='major', linestyle='-', linewidth='0.5', color='black') # customise major grid
ax2.set_axisbelow(True) # Don't allow the axis to be on top of your data, don't necessarily need this line
ax2.minorticks_on() # need this line in order to get the minor grid lines 
ax2.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# change axis labels
ax2.set(xlabel=None)
ax2.set(ylabel=None)
# set axis limits
ax2.set_ylim(ymin= -0.05, ymax=0.07)

# subplot 3 JJA
pr_anom_jja.mean(dim=('lat','lon')).plot(ax=ax3, color='blue')
# add gridlines
ax3.grid(which='major', linestyle='-', linewidth='0.5', color='black') # customise major grid
ax3.set_axisbelow(True) # Don't allow the axis to be on top of your data, don't necessarily need this line
ax3.minorticks_on() # need this line in order to get the minor grid lines 
ax3.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# change axis labels
ax3.set(xlabel=None)
ax3.set(ylabel='Precipitation anomaly [mm/day]')
# set axis limits
ax3.set_ylim(ymin= -0.05, ymax=0.07)

# subplot 4 SON
pr_anom_son.mean(dim=('lat','lon')).plot(ax=ax4, color='green')
# add gridlines
ax4.grid(which='major', linestyle='-', linewidth='0.5', color='black') # customise major grid
ax4.set_axisbelow(True) # Don't allow the axis to be on top of your data, don't necessarily need this line
ax4.minorticks_on() # need this line in order to get the minor grid lines 
ax4.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# change axis labels
ax4.set(ylabel=None)
ax4.set(xlabel=None)
# set axis limits
ax4.set_ylim(ymin= -0.05, ymax=0.07)


f.suptitle('Seasonal precipitation anomalies', fontsize=16)
#f.set_figheight(5) # these two parameters change the figure height and width 
#f.set_figwidth(15)

In [ ]:
# seasonal temperature anomalies from 1850-2014 using a climatology from 1850-1900

f = plt.figure(figsize=(15,10))
ax = f.add_subplot(221)
ax2 = f.add_subplot(222)
ax3 = f.add_subplot(223)
ax4 = f.add_subplot(224)

#subplot 1 DJF
ts_anom_djf.mean(dim=('lat','lon')).plot(ax=ax, color ='red')
# add gridlines
ax.grid(which='major', linestyle='-', linewidth='0.5', color='black') # customise major grid
ax.set_axisbelow(True) # Don't allow the axis to be on top of your data, don't necessarily need this line
ax.minorticks_on() # need this line in order to get the minor grid lines 
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# change axis labels
ax.set(xlabel=None)
ax.set(ylabel='Temperature anomaly [C]')
# set axis limits
ax.set_ylim(ymin = -0.7, ymax = 1.7)
#ax.set_xlim(self, bottom=None, top=None, emit=True, auto=False, *, ymin=None, ymax=None)

# subplot 2 MAM
ts_anom_mam.mean(dim=('lat','lon')).plot(ax=ax2, color='orange')
# add gridlines
ax2.grid(which='major', linestyle='-', linewidth='0.5', color='black') # customise major grid
ax2.set_axisbelow(True) # Don't allow the axis to be on top of your data, don't necessarily need this line
ax2.minorticks_on() # need this line in order to get the minor grid lines 
ax2.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# change axis labels
ax2.set(xlabel=None)
ax2.set(ylabel=None)
# set axis limits
ax2.set_ylim(ymin = -0.7, ymax = 1.7)

# subplot 3 JJA
ts_anom_jja.mean(dim=('lat','lon')).plot(ax=ax3, color='blue')
# add gridlines
ax3.grid(which='major', linestyle='-', linewidth='0.5', color='black') # customise major grid
ax3.set_axisbelow(True) # Don't allow the axis to be on top of your data, don't necessarily need this line
ax3.minorticks_on() # need this line in order to get the minor grid lines 
ax3.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# change axis labels
ax3.set(xlabel=None)
ax3.set(ylabel='Temperature anomaly [C]')
# set axis limits
ax3.set_ylim(ymin = -0.7, ymax = 1.7)

# subplot 4 SON
ts_anom_son.mean(dim=('lat','lon')).plot(ax=ax4, color='green')
# add gridlines
ax4.grid(which='major', linestyle='-', linewidth='0.5', color='black') # customise major grid
ax4.set_axisbelow(True) # Don't allow the axis to be on top of your data, don't necessarily need this line
ax4.minorticks_on() # need this line in order to get the minor grid lines 
ax4.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# change axis labels
ax4.set(ylabel=None)
ax4.set(xlabel=None)
# set axis limits
ax4.set_ylim(ymin = -0.7, ymax = 1.7)


f.suptitle('Seasonal temperature anomalies', fontsize=16)
#f.set_figheight(5) # these two parameters change the figure height and width 
#f.set_figwidth(15)